<a href="https://colab.research.google.com/github/Shtepser/Manyinterpret/blob/main/Day2/%D0%A2%D1%8E%D0%BC%D0%B5%D0%BD%D1%8C_BERT_%D1%8F%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 65.5 MB/s 
     |████████████████████████████████| 101 kB 9.7 MB/s 
     |████████████████████████████████| 6.6 MB 44.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


В современной компьютерной лингвистике вычисление вероятности текста производится в основном за счёт нейронных, а не энграммных моделей. Существует много разновидностей архитектур, мы языковую модель с пропусками (MLM) `BERT`.

## Моделирование с пропусками: BERT

In [ ]:
from transformers import AutoTokenizer, BertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = BertForMaskedLM.from_pretrained("bert-base-cased").to("cuda")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


В модели БЕРТ часть слов можно заменять масками. Модель предсказывает распределение вероятности для каждой позиции (в том числе для слов, присутствующих в тексте).

In [ ]:
 sentences = [
     "Yesterday, all my troubles seemed so far away.",
     "Yesterday, all my [MASK] seemed so far away.",
     "Several space rockets work on dymetylhydrasin.",
     "The Starship prototype descended under active aerodynamic control, accomplished by four vehicles."
 ]
 tokenization = tokenizer(sentences)
 for sentence in tokenization["input_ids"]:
     print(*sentence)
     print(tokenizer.convert_ids_to_tokens(sentence))
     print("")

101 21997 117 1155 1139 16115 1882 1177 1677 1283 119 102
['[CLS]', 'Yesterday', ',', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away', '.', '[SEP]']

101 21997 117 1155 1139 103 1882 1177 1677 1283 119 102
['[CLS]', 'Yesterday', ',', 'all', 'my', '[MASK]', 'seemed', 'so', 'far', 'away', '.', '[SEP]']

101 3728 2000 18437 1250 1113 173 26911 2340 1233 7889 7412 10606 119 102
['[CLS]', 'Several', 'space', 'rockets', 'work', 'on', 'd', '##yme', '##ty', '##l', '##hy', '##dra', '##sin', '.', '[SEP]']

101 1109 6200 3157 8933 9026 1223 2327 170 10771 24662 1654 117 8587 1118 1300 4011 119 102
['[CLS]', 'The', 'Stars', '##hip', 'prototype', 'descended', 'under', 'active', 'a', '##ero', '##dynamic', 'control', ',', 'accomplished', 'by', 'four', 'vehicles', '.', '[SEP]']



In [ ]:
tokenizer.special_tokens_map

{'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'pad_token': '[PAD]',
 'sep_token': '[SEP]',
 'unk_token': '[UNK]'}

Напишем функцию, которая находит самые вероятные слова на месте маски.

In [ ]:
import torch

def probable_words(sentence, tokenizer, model, k=10):
    if sentence.count("[MASK]") != 1:
        raise ValueError("Маска должна быть ровно одна.")
    tokenization = tokenizer(sentence)["input_ids"]
    index = tokenization.index(tokenizer.mask_token_id)
    # tensor: 1 * L
    tensor = torch.LongTensor([tokenization]).to("cuda")
    with torch.no_grad():
        # logits: L * K
        logits = model(tensor)["logits"][0]
    probs = torch.softmax(logits, dim=-1)
    # top_probs: k
    top_probs, top_indexes = torch.topk(probs[index], dim=-1, k=k)
    return tokenizer.convert_ids_to_tokens(top_indexes), top_probs

In [ ]:
sentence = "The Starship prototype descended under active aerodynamic [MASK], accomplished by four vehicles."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

testing:0.323
##s:0.114
tests:0.074
control:0.066
development:0.033
research:0.032
modifications:0.013
conditions:0.012
design:0.011
studies:0.010


In [ ]:
sentence = "Yesterday, all my [MASK] seemed so far away."
top_tokens, top_probs = probable_words(sentence, tokenizer, model, k=20)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

thoughts:0.302
dreams:0.266
memories:0.105
fears:0.028
worries:0.027
friends:0.020
life:0.017
emotions:0.014
feelings:0.013
days:0.009
nightmares:0.007
things:0.007
problems:0.005
troubles:0.005
visions:0.004
parents:0.004
senses:0.004
classes:0.004
family:0.004
concerns:0.003


BERT также токенизует на сабтокены, а не слова.

In [ ]:
sentence = "The Starship prototype descended under active aerodynamic control, accomplished by four vehicles."
token_indexes = tokenizer(sentence)["input_ids"]
tokens = tokenizer.convert_ids_to_tokens(token_indexes)
for index, token in zip(token_indexes, tokens):
    print(index, token)

101 [CLS]
21997 Yesterday
117 ,
1155 all
1139 my
168 _
1882 seemed
1177 so
1677 far
1283 away
119 .
102 [SEP]


Соответственно, чтобы найти вероятность слова aerodynamic, нужно перемножить вероятности трёх сабтокенов разбиения.

In [ ]:
def find_word_probability(sentence, word, tokenizer, model):
    """
    sentence -- предложение, содержащее ровно один символ "_", обозначающий пропуск.
    """
    if sentence.count("_") != 1:
        raise ValueError("Предложение должно содержать ровно один пропуск.")
    masked_sentence = sentence.replace("_", tokenizer.mask_token)
    masked_tokenization = tokenizer(masked_sentence)["input_ids"]
    word_tokenization = tokenizer(word, add_special_tokens=False)["input_ids"]
    word_length = len(word_tokenization)
    index = masked_tokenization.index(tokenizer.mask_token_id)
    masked_tokenization[index:index+1] = [tokenizer.mask_token_id] * word_length
    batch = torch.LongTensor([masked_tokenization]).to("cuda")
    with torch.no_grad():
        logits = model(batch)["logits"][0]
    log_probs = torch.log_softmax(logits[index:index+word_length], dim=-1).cpu().numpy()
    subtoken_log_probs = log_probs[np.arange(word_length), word_tokenization]
    total_prob = subtoken_log_probs.sum()
    return {"total_log_prob": total_prob, "subtoken_probs": np.exp(subtoken_log_probs)}

In [ ]:
import numpy as np
np.set_printoptions(precision=3)

texts = [
    "Yesterday, all my _ seemed so far away.", 
    "The Starship prototype descended under active _ control, accomplished by four vehicles."
] 
words = ["troubles", "aerodynamic"]
for text, word in zip(texts, words):
    print(text, word)
    print(find_word_probability(text, word, tokenizer, model))

Yesterday, all my _ seemed so far away. troubles
{'total_log_prob': -5.313094, 'subtoken_probs': array([0.005], dtype=float32)}
The Starship prototype descended under active _ control, accomplished by four vehicles. aerodynamic
{'total_log_prob': -28.819866, 'subtoken_probs': array([5.661e-04, 1.348e-05, 3.993e-05], dtype=float32)}


На самом деле BERT неадекватно вычисляет вероятности, когда присутствует несколько масок подряд. Поэтому маскировать сабтокены следует слева направо, чтобы при вычислении вероятности последнего сабтокена все остальные были уже заполнены.

Это более точно соответствует формуле условной вероятности:
$$
p(t_1 ... t_k | \mathrm{context}) = p(t_1 | \mathrm{context}) p(t_2 | \mathrm{context}, t_1) ... p(t_k | \mathrm{context}, t_1, ..., t_{k-1})
$$

In [ ]:
def find_word_probability(sentence, word, tokenizer, model):
    """
    sentence -- предложение, содержащее ровно один символ "_", обозначающий пропуск.
    """
    if sentence.count("_") != 1:
        raise ValueError("Предложение должно содержать ровно один пропуск.")
    masked_sentence = sentence.replace("_", tokenizer.mask_token)
    masked_tokenization = tokenizer(masked_sentence)["input_ids"]
    word_tokenization = tokenizer(word, add_special_tokens=False)["input_ids"]
    word_length = len(word_tokenization)
    index = masked_tokenization.index(tokenizer.mask_token_id)
    masked_tokenization[index:index+1] = [tokenizer.mask_token_id] * word_length
    # повторяем маскированную токенизацию `word_length` раз
    # [[active MASK MASK MASK control] [active MASK MASK MASK control] [active MASK MASK MASK control]]
    batch = np.array([masked_tokenization] * word_length, dtype=int)
    # заполняем начальные сабтокены
    for prefix_length in range(1, word_length):
        '''[
            [active MASK(a) MASK MASK control] 
            [active a MASK(ero) MASK control] 
            [active a ero MASK(dynamic) control]
        ]'''
        batch[prefix_length, index:index+prefix_length] = word_tokenization[:prefix_length]
    batch = torch.LongTensor(batch).to("cuda")
    with torch.no_grad():
        logits = model(batch)["logits"]
    # в первом примере нам нужна позиция с номером index, далее index+1, ...
    print(logits.shape)
    log_probs = torch.log_softmax(
        logits[np.arange(word_length),index+np.arange(word_length)], dim=-1
    ).cpu().numpy()
    print(log_probs.shape)
    subtoken_log_probs = log_probs[np.arange(word_length), word_tokenization]
    total_prob = subtoken_log_probs.sum()
    return {"total_log_prob": total_prob, "subtoken_probs": np.exp(subtoken_log_probs)}

In [ ]:
np.set_printoptions(precision=3)

texts = [
    "Yesterday, all my _ seemed so far away.", 
    "The Starship prototype descended under active _ control, accomplished by four vehicles."
] 
words = ["troubles", "aerodynamic"]
for text, word in zip(texts, words):
    print(text, word)
    print(find_word_probability(text, word, tokenizer, model))

Yesterday, all my _ seemed so far away. troubles
torch.Size([1, 12, 28996])
(1, 28996)
{'total_log_prob': -5.313094, 'subtoken_probs': array([0.005], dtype=float32)}
The Starship prototype descended under active _ control, accomplished by four vehicles. aerodynamic
torch.Size([3, 19, 28996])
(3, 28996)
{'total_log_prob': -11.110754, 'subtoken_probs': array([5.661e-04, 2.942e-02, 8.977e-01], dtype=float32)}


В целом вероятностям БЕРТа не стоит слишком доверять, когда они вычисляются для слов, состоящих из нескольких подтокенов.

### Модель для русского языка

In [ ]:
from transformers import AutoTokenizer, BertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased").to("cuda")

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentence = "Все счастливые семьи похожи друг на друга, каждая несчастная семья несчастлива по-своему."
token_indexes = tokenizer(sentence)["input_ids"]
tokens = tokenizer.convert_ids_to_tokens(token_indexes)
for index, token in zip(token_indexes, tokens):
    print(index, token)

101 [CLS]
10351 Все
108712 счастливы
842 ##е
12060 семьи
40611 похожи
3487 друг
1469 на
15200 друга
128 ,
22780 каждая
26026 несчаст
2630 ##ная
15212 семья
26026 несчаст
36289 ##лива
1516 по
130 -
15008 своему
132 .
102 [SEP]


In [ ]:
sentence = "Что такое [MASK], это небо, плачущее небо под ногами."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

небо:0.465
солнце:0.049
море:0.028
земля:0.027
любовь:0.020
вода:0.008
музыка:0.007
луна:0.006
ветер:0.006
жизнь:0.006


Проверим, как модель знает грамматику и другие аспекты языка.

In [ ]:
sentence = "Съеденное вчера пирожное показалось ему очень [MASK] ."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

интересным:0.202
красивым:0.094
хорошим:0.073
странным:0.053
неприятным:0.053
подозрительным:0.041
ценным:0.037
слабым:0.028
необычным:0.026
сильным:0.022


In [ ]:
sentence = "Съеденная булка показалась ему очень [MASK] , так что он купил ещё одну ."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

интересной:0.188
странной:0.073
удачной:0.061
привлекательной:0.041
полезной:0.036
хорошей:0.032
дорогой:0.032
опасной:0.029
удобной:0.028
подходящей:0.027


In [ ]:
sentence = "Съеденная булка показалась ему очень [MASK] , так что он потребовал деньги назад ."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

интересной:0.181
странной:0.076
привлекательной:0.052
опасной:0.042
удобной:0.040
дорогой:0.029
плохо:0.026
ценной:0.025
хорошей:0.024
удачной:0.023


Как мы видим, модель хорошо справляется с согласованием по роду, но куда хуже с пониманием тональности и смысла текста.